<a href="https://colab.research.google.com/github/champsleague/DeepLearning/blob/main/DL_Lab06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Import Dependencies

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torchvision.datasets as datasets
import torchvision.transforms as transforms


# Device Definition 

In [6]:
use_cuda = torch.cuda.is_available() 
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


# Define Hyperparameters

In [7]:
batch_size = 128
print_train_step = int(1000*16/batch_size)
#learnning_rate = 0.01
learnning_rate = 0.02
epochs = 10

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Downloading & Loading MNIST data

In [8]:
train_dataset = datasets.MNIST(
    './data', train=True, download=True,
    transform=transforms.Compose([
                                  transforms.Resize([112, 112]),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))
                                  ])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)

test_dataset = datasets.MNIST(
    './data', train=False, 
    transform=transforms.Compose([
                                  transforms.Resize([112, 112]),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))
                                  ])
    )

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, **kwargs)

print('number of training data : ',len(train_dataset))
print('number of test data : ',len(test_dataset))

100%|██████████| 9912422/9912422 [00:00<00:00, 89965012.69it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 44765592.69it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25795362.27it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4348443.00it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

number of training data :  60000
number of test data :  10000


# Define model class (Transfer Learning)

In [9]:
model = models.resnet18(pretrained=True)
print(model)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 207MB/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Fine-tuning


In [16]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride = (1, 1), padding = (1, 1), bias=False)
model.fc = nn.Linear(512, 10, bias = True)

model.to(device)

for name, param in model.named_parameters():
  param.requires_grad = False

model.conv1.weight.requires_grad = True
model.bn1.weight.requires_grad = True
model.bn1.bias.requires_grad = True
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True

# params_to_update = []
# for name, param in model.named_parameters():
#   if param.requires_grad == True:
#     print(name)
#     params_to_update.append(param)  

# print(model)

params_to_update = []
for name, param in model.named_parameters():
  if 'conv1' in name:
    print(name)    
    param.requires_grad = True
    params_to_update.append(param)
  elif 'fc' in name:
    print(name)    
    param.requires_grad = True
    params_to_update.append(param)
  else:
    param.requires_grad = False

print(model)

conv1.weight
layer1.0.conv1.weight
layer1.1.conv1.weight
layer2.0.conv1.weight
layer2.1.conv1.weight
layer3.0.conv1.weight
layer3.1.conv1.weight
layer4.0.conv1.weight
layer4.1.conv1.weight
fc.weight
fc.bias
ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1):

# Define Loss Function & Optimizer

In [17]:
optimizer = optim.Adadelta(params_to_update, lr=learnning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
loss_function = nn.CrossEntropyLoss()

# Define Model Training

In [18]:
def train(model, device, train_loader, optimizer, loss_function, epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = loss_function(output, target)
    loss.backward()
    optimizer.step()
        
    if batch_idx % print_train_step == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))


# Define Model Evaluation

In [19]:
def test(model, device, test_loader, loss_function):
  model.eval()
  test_loss = 0
  correct = 0
  
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += loss_function(output, target) # sum up batch loss
      pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
      correct += pred.eq(target.view_as(pred)).sum().item()
      
  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))


# Train the Model


In [20]:
for epoch in range(epochs):
  train(model, device, train_loader, optimizer, loss_function, epoch)
  test(model, device, test_loader, loss_function)
  scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.582134
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.351808
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.190158
Train Epoch: 0 [48000/60000 (80%)]	Loss: 0.118841

Test set: Average loss: 0.0006, Accuracy: 9864/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.048148
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.072598
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.087667
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.074559

Test set: Average loss: 0.0005, Accuracy: 9874/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.035554
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.069648
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.097860
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.084527

Test set: Average loss: 0.0005, Accuracy: 9872/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.052346
Train Epoch: 3 [16000/60000 (27%)]	Loss: 0.072294
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.050788
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.036846

Test set: Average loss: 0.0005, Accu

# Hint --> 98% Model (Fine-tuning)

In [15]:
params_to_update = []
for name, param in model.named_parameters():
  if 'conv1' in name:
    print(name)    
    param.requires_grad = True
    params_to_update.append(param)
  elif 'fc' in name:
    print(name)    
    param.requires_grad = True
    params_to_update.append(param)
  else:
    param.requires_grad = False

print(model)

conv1.weight
layer1.0.conv1.weight
layer1.1.conv1.weight
layer2.0.conv1.weight
layer2.1.conv1.weight
layer3.0.conv1.weight
layer3.1.conv1.weight
layer4.0.conv1.weight
layer4.1.conv1.weight
fc.weight
fc.bias
ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1):